In [ ]:
from platform import python_version
import platform
print(dir(platform))

In [ ]:
platform.python_version()

In [12]:
from selenium import webdriver
import re
from bs4 import BeautifulSoup
import random
import time
import pandas as pd
import datetime
import nltk

In [2]:
# key words
key_words = [
    '"edemocracia"',
    'democracia digital',
    'democracia eletrônica',
    'democracia virtual',
    'democracia online',
    'democracia AND novas mídias',
    'democracia AND tecnologia',
    'ciberdemocracia',
    'teledemocracia',
    'democracia AND Internet',
    # participação OK
    'eparticipação',
    'participação AND tecnologia',
    'participação AND Internet',
    'participação AND digital',
    'participação AND virtual',
    'participação AND TICs',
    'participação AND novas mídias',
    'participação AND online',
    'participação política online',
    'participação política digital',
    'participação política virtual',
    # deliberação OK
    'edeliberação',
    'Deliberação AND Internet',
    'Deliberação AND digital',
    '"Deliberação virtual"',
    '"Deliberação online"',
    # esfera puública
    'Esfera Pública & Internet',
    'Esfera Pública & digital',
    'Esfera Pública virtual',
    'Esfera Pública online',
    # transparência
    'etransparência',
    'transparência digital',
    'transparência AND novas mídias',
    'transparência online',
    'transparência AND internet',
    'Transparência AND tecnologia',
    'Transparência AND TICs',
    'transparência AND websites',
    'Transparência AND portais',
    'governo aberto',
    'Dados abertos governamentais',
    'parlamento aberto',
    'parlamento digital',
    'webcidadania',
    'exclusão digital'
]

In [17]:
def getListFromGrid(soup):
    artigos = []
    elements = soup.findAll("div", {"class":"gs_r gs_or gs_scl"})
    for element in elements:
        # title
        try:
            title = element.find("h3", {"class":"gs_rt"})
            title_text = title.text
        except:
            title_text = "NA"
        
        # title_url
        try:
            title_url = title.find("a", href=True)
            title_url = title_url["href"]
        except:
            title_url = "NA"
        
        # doc_url
        try:
            document_url = element.find("div", {"class":"gs_ggs gs_fl"})
            document_url = document_url.find("a", href=True)
            document_url = document_url["href"]
        except:
            document_url = "NA"

        # green_line
        try:
            green_line = element.find("div", {"class":"gs_a"})
            green_line
        except:
            green_line = "NA"

        # resume
        try:
            abstract = element.find("div", {"class":"gs_rs"})
            abstract_text = abstract.text
        except:
            abstract_text = "NA"

        artigos.append({"title":title_text,
                        "title_url":title_url,
                        "doc_url":document_url,
                        "green_line":green_line,
                        "abstract":abstract_text})

    return artigos

In [ ]:
a = '"democracia" OR democracia digital OR "democracia eletrônica"  OR "democracia virtual" OR "democracia online" OR democracy AND novas mídias democracia AND tecnologia OR ciberdemocracia OR  teledemocracia OR democracia AND Internet'
a = re.sub('\s+', '+', a)
a

In [ ]:
# esse aqui

#
url = 'https://scholar.google.com.br/scholar?start={}&q="democracia"+OR+democracia+digital+OR+"democracia+eletrônica"+OR+"democracia+virtual"+OR+"democracia+online"+OR+democracy+AND+novas+mídias+democracia+AND+tecnologia+OR+ciberdemocracia+OR+teledemocracia+OR+democracia+AND+Internet&btnG='
driver = webdriver.Chrome("/home/victor/Documents/chromedriver")

grids = []
# FIRST
driver.get('https://scholar.google.com.br/')
time.sleep(13)
driver.get(url.format(0))
time.sleep((random.random()*10)/3)
grid = driver.find_element_by_css_selector("#gs_res_ccl_mid")
grids.append(grid.get_attribute("innerHTML"))
time.sleep((random.random()*10)/2)
#/ FIRST

nResults = driver.find_element_by_css_selector("#gs_ab_md > div")
nResults = nResults.text
nResults = nResults.split("results")[0]
nResults = [e for e in nResults if e.isdigit()]
nResults = "".join(nResults).strip()
nResults = round((int(nResults) + 10)/10)
print(nResults)
for i in range(1, 100):

    driver.get(url.format(i*10))
    time.sleep((random.random()*10)/2)

    grid = driver.find_element_by_css_selector("#gs_res_ccl_mid")
    grids.append(grid.get_attribute("innerHTML"))
    time.sleep((random.random()*10)/2)

    print(i*10)



In [19]:
# write grids
for i in range(len(grids)):
    file = open("/home/victor/Desktop/grids/{}.html".format(i), "w+")
    file.write(grids[i])
    file.close()

In [20]:
# grids to lists
artigos = []
for grid in grids:
    grid = BeautifulSoup(grid)
    grid_list = getListFromGrid(grid)
    artigos.extend(grid_list)

In [21]:
# lists to DF
papers_df = pd.DataFrame(data={
    "title_text":[e["title"] for e in artigos],
    "title_url":[e["title_url"] for e in artigos],
    "green_line":[e["green_line"] for e in artigos],
    "doc_url":[e["doc_url"] for e in artigos],
    "abstract":[e["abstract"] for e in artigos]
})

now = datetime.datetime.strftime(datetime.datetime.now(), "%d-%m-%Y %Hh%M")
papers_df.to_csv("/home/victor/Desktop/papers {}.csv".format(now), index=False)

In [ ]:
list(new_papers_df.columns.values)["index"]

In [ ]:
new_papers_df = papers_df.loc[ ["[HTML]" not in e for e in papers_df["title_text"]]]
new_papers_df = new_papers_df.loc[ ["[CITAÇÃO]" not in e for e in new_papers_df["title_text"]]]
new_papers_df = new_papers_df.loc[ ["[BOOK]" not in e for e in new_papers_df["title_text"]]]
new_papers_df = new_papers_df.loc[ ["[LIVRO]" not in e for e in new_papers_df["title_text"]]]
new_papers_df = new_papers_df.reset_index()
new_papers_df = new_papers_df.loc[:,new_papers_df.columns != "index"]
new_papers_df

In [ ]:
titles_url = list(papers_df['title_url'])
sites_list = []
for e in titles_url:
    try:
        site = re.search(".*?\.(.*?\..*?)\.", e).group(1)
        sites_url = sites_list.append(site)
    except:
        sites_url = sites_list.append("NA")

unique_sites = list(set(sites_list))

for u in unique_sites:
    print(u)
    
print(len(unique_sites))

count_sites = {}
for u in unique_sites:
    count_sites[u] = sites_list.count(u)

In [ ]:
keys = list(count_sites.keys())
values = list(count_sites.values())

sites_count_df = pd.DataFrame(data={"keys":keys, "values":values})
sites_count_df

In [30]:
en = len(new_papers_df)
l = en/6
new_papers_df['delete'] = 0
new_papers_df.loc[1:l, 'classificador'] = 'victor'
new_papers_df.loc[l:l*2, 'classificador'] = 'murillo'
new_papers_df.loc[l*2:l*3, 'classificador'] = 'tiago'
new_papers_df.loc[l*3:l*4, 'classificador'] = 'pri'
new_papers_df.loc[l*4:l*5, 'classificador'] = 'pedro'
new_papers_df.loc[l*5:en, 'classificador'] = 'rafael'
new_papers_df.to_csv('FUNCIONOU AEHOO.csv', index=False)

In [ ]:
len(new_papers_df)